In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor
%cd /content/keras_triplet_descriptor
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip



RAM Free: 7.4 GB  | Proc size: 4.5 GB
GPU RAM Free: 8511MB | Used: 2930MB | Util  26% | Total 11441MB
Cloning into 'keras_triplet_descriptor'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 181 (delta 0), reused 1 (delta 0), pack-reused 178
Receiving objects: 100% (181/181), 149.87 MiB | 18.38 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (69/69), done.
/content/keras_triplet_descriptor
--2019-03-22 22:55:10--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 107.152.26.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-22 22:55:10--  https:/

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

import psutil
import humanize
import os
import GPUtil as GPU

import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))

In [0]:
def get_denoise_model(shape):
    
  init = keras.initializers.he_normal(seed = 10)
  
  inputs = Input(shape)
  
  conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)




  ## Now the decoder starts
  up6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(pool3)


  up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = init)(UpSampling2D(size = (2,2))(up6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(merge7)

  up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = init)(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(merge8)

  up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = init)(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = init)(merge9)
  conv10 = Conv2D(1, 3,  padding = 'same')(conv9)

  model = Model(inputs = inputs, outputs = conv10)
  
  return model




def get_descriptor_model(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init = keras.initializers.truncated_normal(seed = 20)
  
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  

#  descriptor_model.add(UpSampling2D(size = (2,2)))
  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model
  
  
def triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

In [0]:

denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)

shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)

100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


In [0]:

opt_dn = keras.optimizers.Adamax(lr=0.0025, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
denoise_model.compile(loss='mean_absolute_error', optimizer=opt_dn, metrics=['mae'])
epochs = 20




    
    
denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=epochs, verbose=1, 
                                                validation_data=denoise_generator_val)

              

  

        



Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1101/1101 [==============================] - 54s 49ms/step - loss: 15.2147 - mean_absolute_error: 15.2147 - val_loss: 7.2135 - val_mean_absolute_error: 7.2135
Epoch 2/20
1101/1101 [==============================] - 52s 47ms/step - loss: 6.5717 - mean_absolute_error: 6.5717 - val_loss: 6.6955 - val_mean_absolute_error: 6.6955
Epoch 3/20
1101/1101 [==============================] - 52s 47ms/step - loss: 6.2152 - mean_absolute_error: 6.2152 - val_loss: 6.3347 - val_mean_absolute_error: 6.3347
Epoch 4/20
1101/1101 [==============================] - 52s 47ms/step - loss: 6.0302 - mean_absolute_error: 6.0302 - val_loss: 6.1924 - val_mean_absolute_error: 6.1924
Epoch 5/20
1101/1101 [==============================] - 51s 47ms/step - loss: 5.8103 - mean_absolute_error: 5.8103 - val_loss: 6.0991 - val_mean_absolute_error: 6.0991
Epoch 6/20
1101/1101 [==============================] - 51s 47ms/step - loss: 5.7293 - mean_absolute_error: 5.

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer='adam')

In [0]:
### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=denoise_model, use_clean=False)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

Using denoised patches
 23%|██▎       | 27/116 [00:10<00:40,  2.22it/s]

In [0]:

epochs = 20


    
for e in range(epochs):
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)




In [0]:
descriptor_model = load_model('desc_final.h5')

generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)


100%|██████████| 40/40 [14:54<00:00, 22.14s/it]


In [0]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval


>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
>> Descriptor files loaded.
>> Evaluating verification task
Processing verification task 1/3 : 100% 1000000/1000000 [07:03<00:00, 2359.38it/s]
Processing verification task 2/3 : 100% 1000000/1000000 [07:29<00:00, 2223.95it/s]
Processing verification task 3/3 : 100% 1000000/1000000 [07:35<00:00, 2193.80it/s]
>> Verification task finished in 1339 secs  
Verification task results:
Mean Average Precision is 0.455371

>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some job

KeyboardInterrupt: ignored